# 使用底层api

为了搞清楚怎么使用余弦相似度。

从底层api实现查询。

In [1]:
%%time

import logging
import sys

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core import StorageContext
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from IPython.display import Markdown, display
from llama_index.core import Settings
from llama_index.embeddings.ollama import OllamaEmbedding

from llama_index.llms.openai_like import OpenAILike

from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.vector_stores.milvus import MilvusVectorStore

CPU times: user 3.49 s, sys: 543 ms, total: 4.03 s
Wall time: 3.69 s


In [2]:
%%time

# llm
llm = OpenAILike(model="qwen14b", 
                 api_base="http://192.168.0.72:3000/v1", 
                 api_key="sk-bJP6QSnUfjAYeYeE505d3eBf63A643BeB0B8E350Df9b7750",
                 is_chat_model=True,
                 temperature=0.1
                )

CPU times: user 142 µs, sys: 0 ns, total: 142 µs
Wall time: 145 µs


In [3]:
%%time

# embedding
from llama_index.embeddings.ollama import OllamaEmbedding

ollama_embedding = OllamaEmbedding(
    model_name="dztech/bge-large-zh:v1.5",
    # model_name="bge-m3:latest",
    base_url="http://192.168.0.72:11435",
    ollama_additional_kwargs={"mirostat": 0}, # -mirostat N 使用 Mirostat 采样。
)

CPU times: user 105 µs, sys: 0 ns, total: 105 µs
Wall time: 109 µs


In [4]:
Settings.chunk_size=128
Settings.chunk_overlap=10

Settings.llm =llm
Settings.embed_model = ollama_embedding

Settings

_Settings(_llm=OpenAILike(callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7fac5775d420>, system_prompt=None, messages_to_prompt=<function messages_to_prompt at 0x7fac624d8820>, completion_to_prompt=<function default_completion_to_prompt at 0x7fac62517010>, output_parser=None, pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'>, query_wrapper_prompt=None, model='qwen14b', temperature=0.1, max_tokens=None, logprobs=None, top_logprobs=0, additional_kwargs={}, max_retries=3, timeout=60.0, default_headers=None, reuse_client=True, api_key='sk-bJP6QSnUfjAYeYeE505d3eBf63A643BeB0B8E350Df9b7750', api_base='http://192.168.0.72:3000/v1', api_version='', context_window=3900, is_chat_model=True, is_function_calling_model=False, tokenizer=None), _embed_model=OllamaEmbedding(model_name='dztech/bge-large-zh:v1.5', embed_batch_size=10, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7fac5775d420>, num_workers=None, base_url='http://1

In [5]:
%%time

from milvus import default_server
from pymilvus import connections, utility

default_server.set_base_dir('milvus_data')

# (OPTIONAL) if you want cleanup previous data
default_server.cleanup()

# Start your milvus server
default_server.start()

# Now you could connect with localhost and the given port
# Port is defined by default_server.listen_port
connections.connect(host='127.0.0.1', port=default_server.listen_port)

# Check if the server is ready.
print(utility.get_server_version())

v2.3.5-lite
CPU times: user 3.18 s, sys: 272 ms, total: 3.45 s
Wall time: 6.76 s


In [6]:
%%time

# load documents
documents = SimpleDirectoryReader(input_files=["./books1/故乡.txt"]).load_data()
print("Document ID:", documents[0].doc_id)

Document ID: 00b4ba0f-12f8-4d57-8bcb-a4a975760d78
CPU times: user 10.5 ms, sys: 53 µs, total: 10.6 ms
Wall time: 10.2 ms


In [7]:
%%time

vector_store = MilvusVectorStore(dim=1024, overwrite=True)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context
)

CPU times: user 400 ms, sys: 20 ms, total: 420 ms
Wall time: 12.7 s


In [8]:
%%time

index.storage_context.persist()

CPU times: user 1.79 ms, sys: 0 ns, total: 1.79 ms
Wall time: 1.59 ms


In [14]:
%%time

from llama_index.core import VectorStoreIndex, get_response_synthesizer
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine

# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=3,
)

# configure response synthesizer
response_synthesizer = get_response_synthesizer(
    response_mode="tree_summarize",
    streaming=True,
)

# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
)


CPU times: user 157 µs, sys: 0 ns, total: 157 µs
Wall time: 160 µs


In [15]:
%%time

streaming_response = query_engine.query("闰土是谁")
streaming_response.print_response_stream()
print()

闰土是小说《故乡》中的人物，他是主人公鲁迅（即文中的我）儿时的朋友，和我同年出生，因为生于闰月，五行缺土，所以他的父亲给他取名为闰土。闰土以能装弓捉小鸟雀而闻名。在故事中，他来管祭器，并且与"我"见面时表现得有些拘谨和害羞。

CPU times: user 115 ms, sys: 26.6 ms, total: 142 ms
Wall time: 3.61 s


In [16]:
for node in streaming_response.source_nodes:
    print(node.score,node.metadata["file_name"],(lambda s: s.replace('\n', '')[:20] + '..' if len(s.replace('\n', '')) > 10 else s.replace('\n', ''))(node.text))

227.4545440673828 故乡.txt 我这时很兴奋，但不知道怎么说才好，只是说..
218.80593872070312 故乡.txt 闰土来管祭器的。　　我的父亲允许了；我也..
216.45843505859375 故乡.txt “阿呀，老太太真是……这成什么规矩。那时..


In [17]:
%%time

from llama_index.core.postprocessor import SentenceTransformerRerank

reranker = SentenceTransformerRerank(model='/models/bge-reranker-v2-m3', top_n=5)

CPU times: user 1.74 s, sys: 1.48 s, total: 3.21 s
Wall time: 20.6 s


In [20]:
%%time

query_engine = index.as_query_engine(
    streaming=True,
    similarity_top_k=100,
    node_postprocessors=[reranker]
)

streaming_response = query_engine.query("闰土是谁")
streaming_response.print_response_stream()
print()

闰土是鲁迅笔下小说《故乡》中的人物，他是一个十多岁的小男孩，五行缺土，因此他的父亲给他取名为闰土。他能装弓捉小鸟雀，是小说中的一个重要角色。



CPU times: user 596 ms, sys: 1.15 ms, total: 597 ms
Wall time: 3.9 s


In [21]:
for node in streaming_response.source_nodes:
    print(node.score,node.metadata["file_name"],(lambda s: s.replace('\n', '')[:20] + '..' if len(s.replace('\n', '')) > 10 else s.replace('\n', ''))(node.text))

0.8655411 故乡.txt 腐西施的杨二嫂，自从我家收拾行李以来，本..
0.8528333 故乡.txt 闰土来管祭器的。　　我的父亲允许了；我也..
0.79228044 故乡.txt ，反从他的胯下逃走了。　　这少年便是闰土..
0.63945824 故乡.txt 工的叫长工；按日给人做工的叫短工；自己也..
0.62720495 故乡.txt 一日是天气很冷的午后，我吃过午饭，坐着喝..


In [22]:
streaming_response = query_engine.query("文中提到的孔乙己是谁")
streaming_response.print_response_stream()
print()

《故乡》是鲁迅的一篇小说，文中并没有提及名叫孔乙己的人物。这个查询中的“孔乙己”可能是指鲁迅另一部作品《呐喊》中的人物，但根据给定的文本，我们无法确定。在提供的信息中，提到的是闰土、杨二嫂等人物。


In [23]:
for node in streaming_response.source_nodes:
    print(node.score,node.metadata["file_name"],(lambda s: s.replace('\n', '')[:20] + '..' if len(s.replace('\n', '')) > 10 else s.replace('\n', ''))(node.text))

0.012071134 故乡.txt 发见了这件事，自己很以为功，便拿了那狗气..
0.01149296 故乡.txt 腐西施的杨二嫂，自从我家收拾行李以来，本..
0.010160264 故乡.txt ，反从他的胯下逃走了。　　这少年便是闰土..
0.0077564907 故乡.txt ⑹西施：春秋时越国的美女，后来用以泛称一..
0.00727125 故乡.txt 他正在厨房里，紫色的圆脸，头戴一顶小毡帽..


In [24]:
%%time

from llama_index.core.postprocessor import SimilarityPostprocessor

postprocessor = SimilarityPostprocessor(similarity_cutoff=0.7)

query_engine = index.as_query_engine(
    streaming=True,
    similarity_top_k=100,
    node_postprocessors=[postprocessor]
)

streaming_response = query_engine.query("闰土是谁")
streaming_response.print_response_stream()
print()

闰土是鲁迅先生笔下《故乡》中的一个重要角色，他象征着旧时农村生活的典型代表。
CPU times: user 190 ms, sys: 2.81 ms, total: 193 ms
Wall time: 12.7 s


In [25]:
for node in streaming_response.source_nodes:
    print(node.score,node.metadata["file_name"],(lambda s: s.replace('\n', '')[:20] + '..' if len(s.replace('\n', '')) > 10 else s.replace('\n', ''))(node.text))

227.4545440673828 故乡.txt 我这时很兴奋，但不知道怎么说才好，只是说..
218.80593872070312 故乡.txt 闰土来管祭器的。　　我的父亲允许了；我也..
216.45843505859375 故乡.txt “阿呀，老太太真是……这成什么规矩。那时..
213.39610290527344 故乡.txt “老太太。信是早收到了。我实在喜欢的不得..
212.62362670898438 故乡.txt 工的叫长工；按日给人做工的叫短工；自己也..
211.82582092285156 故乡.txt 他见人很怕羞，只是不怕我，没有旁人的时候..
210.07339477539062 故乡.txt ，反从他的胯下逃走了。　　这少年便是闰土..
209.5423583984375 故乡.txt 一日是天气很冷的午后，我吃过午饭，坐着喝..
209.53453063964844 故乡.txt 我于是日日盼望新年，新年到，闰土也就到了..
209.19378662109375 故乡.txt “我们沙地里，潮汛要来的时候，就有许多跳..
208.85830688476562 故乡.txt 虽然我一见便知道是闰土，但又不是我这记忆..
208.84716796875 故乡.txt ”　　“回来？你怎么还没有走就想回来了。..
208.17913818359375 故乡.txt 什么都有：稻鸡，角鸡，鹁鸪，蓝背……”　..
203.59249877929688 故乡.txt ”便拖出躲在背后的孩子来，这正是一个廿年..
202.58010864257812 故乡.txt 我希望他们不再像我，又大家隔膜起来……然..
201.83547973632812 故乡.txt 夜间，我们又谈些闲天，都是无关紧要的话；..
201.68153381347656 故乡.txt 母亲叫闰土坐，他迟疑了一回，终于就了坐，..
201.03677368164062 故乡.txt 他们不知道一些事，闰土在海边时，他们都和..
200.26431274414062 故乡.txt 我躺着，听船底潺潺的水声，知道我在走我的..
200.21633911132812 故乡.txt “是的。”　　“还有闰土，他每到我家来时..
199.05599975585938 故乡.txt